In [1]:
`# if the files are on Anvil / Terra workspace 
# please run it from the workspace analysis to avoid bucket access permission issues
# or use the gcloud login auth interface to authenticate yourself

In [2]:
!pwd

/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation


installations:

1. pip install synapseclient

In [3]:
import os
import pandas as pd
# CHANGE
gs_path_for_the_encode_pseudobulk_pipeline_table = "ENCODE_PSEUDOBULK_TABLE_INPUT"
local_folder = os.getcwd() #"/home/jupyter/eila" #os.getcwd()
os.makedirs(local_folder, exist_ok=True)
local_metadata_tsv = "proc.tsv"
# NO NEED TO CHANGE:
bucket_id = "fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901"
# if human - no need to change. taken from the worspace dashboard
gs_chrom_size_human = "gs://broad-buenrostro-pipeline-genome-annotations/IGVF_human_v43/GRCh38_EBV.chrom.sizes.tsv"

In [4]:
def remove_file(filename):
  print("remove_file method: {}".format(filename))
  if os.path.exists(filename):
    os.remove(filename)

In [5]:
!ls

clear_notebook_output.ipynb
eila_encode_pseudobulk_fragments_streaming_from_analysis.ipynb
gsutil_ls_txt.txt
proc.tsv
syn46645403_labels
synapse_utils_from_analysis.ipynb
tsv_files_utils_from_analysis.ipynb


In [6]:
%run synapse_utils_from_analysis.ipynb


Welcome, eila!



In [7]:
%run tsv_files_utils_from_analysis.ipynb


In [8]:
# read the proc.tsv object
# Create a subset file of each rna_dataset and save it under the rna_dataset and call it {rna_dataset}_metadata.tsv

# TODO: copy the metadata file for ENCODE datasets
local_metadata_file_location = os.path.join(local_folder,local_metadata_tsv)
if os.path.exists(local_metadata_file_location) == False:
    print("execute cmd: gsutil cp gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/ENCODE_synpase_annotation_mapping/proc.tsv {}".format(local_metadata_file_location))
    !gsutil cp gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/ENCODE_synpase_annotation_mapping/proc.tsv $local_metadata_file_location
else:
    print("file proc.tsv is available, nothing to copy")

file proc.tsv is available, nothing to copy


In [9]:
# !gsutil cp 'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/ENCODE_synpase_annotation_mapping/proc.tsv' .

In [10]:
import csv
from itertools import islice
import pandas as pd
metadata_col_names = None
df_relevant_rows = None


with open(local_metadata_file_location, "r", newline="") as tsvfile:
    reader = csv.reader(tsvfile, delimiter="\t")  # Set delimiter as tab

# Column names: ['cell_type_name,"rna_dataset","rna_library","rna_barcode","atac_dataset","atac_library","atac_barcode","file"']    # Read the header row
    metadata_col_names = next(reader)
    print("Column names:", metadata_col_names," type is {}".format(type(metadata_col_names)))
    metadata_col_names = metadata_col_names[0].split(",")
    print("metadata_col_names after split is {}".format(metadata_col_names))
    metadata_col_names = [x.strip('"') for x in metadata_col_names]
    print("metadata_col_names after strip is {}".format(metadata_col_names))
    print("Column names after processing is:", metadata_col_names," type is {}".format(type(metadata_col_names)))
    column_atac_dataset_index = metadata_col_names.index("atac_dataset")
    print("column_atac_dataset_index is {}".format(column_atac_dataset_index))

    rows = []  # To store the extracted rows
    for row in reader: # to do: un-comment
        # Process each row
        # print("row is {}. type is {}. len is {}".format(row,type(row),len(row)))
        row = row[0].split(",")
        row = [x.strip('"') for x in row]
        column_atac_dataset_value = row[column_atac_dataset_index]
        
        if column_atac_dataset_value != 'NA':
            rows.append(row)
            # print("adding row {}".format(row))
        else:
            # print("Nothing to add: column_atac_dataset_value is {}".format(column_atac_dataset_value))
            continue
    # Create a dataframe from the extracted rows
    df_encode_rows = pd.DataFrame(rows, columns=metadata_col_names)

Column names: ['cell_type_name,"rna_dataset","rna_library","rna_barcode","atac_dataset","atac_library","atac_barcode","file"']  type is <class 'list'>
metadata_col_names after split is ['cell_type_name', '"rna_dataset"', '"rna_library"', '"rna_barcode"', '"atac_dataset"', '"atac_library"', '"atac_barcode"', '"file"']
metadata_col_names after strip is ['cell_type_name', 'rna_dataset', 'rna_library', 'rna_barcode', 'atac_dataset', 'atac_library', 'atac_barcode', 'file']
Column names after processing is: ['cell_type_name', 'rna_dataset', 'rna_library', 'rna_barcode', 'atac_dataset', 'atac_library', 'atac_barcode', 'file']  type is <class 'list'>
column_atac_dataset_index is 4


In [11]:
df_encode_rows.head(2)

,cell_type_name,rna_dataset,rna_library,rna_barcode,atac_dataset,atac_library,atac_barcode,file
0,Hepatocytes,NA,NA,NA,ENCSR114OKU,ENCLB431YMH,GGAGGATCAAGCGGTA,EAID_000002
1,Hepatocytes,NA,NA,NA,ENCSR114OKU,ENCLB431YMH,GTTACGATCGTGGTAT,EAID_000002


In [12]:
len(df_encode_rows['atac_dataset'].to_list())

1236715

In [13]:
# gsutil_ls_txt.txt was generated with gsutils ls gs://path/to/fragmetn/files.tsv.gz
!ls gsutil_ls_txt*

gsutil_ls_txt.txt


In [14]:
# LOAD the ls file from local - does not work on this machine
def load_gs_ls_text_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        lines = [line.strip() for line in lines]
    return lines

# Example usage
file_path = 'gsutil_ls_txt.txt'
gs_path_to_search_sample_id = load_gs_ls_text_file(file_path)
print(gs_path_to_search_sample_id[0:5])

['gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/0e8c4f76-5f9f-4dc8-8ef3-608afb9395ab/share/38144a43-b98a-4c90-93ff-a7fbb88efd2f/call-atac/wf_atac/e7b3e84a-fea9-4fe6-ad7a-40af393fdb36/call-align/ENCSR987PQH.atac.filter.fragments.hg38.tsv.gz', 'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/8ef75ac6-1a49-4cbe-b61d-d62af4c904d7/share/24e3d191-38aa-4ed1-9255-dbfb14161097/call-atac/wf_atac/3d2abad2-1ff0-4477-a7a8-35aef31e2a3e/call-align/ENCSR618WVK.atac.filter.fragments.hg38.tsv.gz', 'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/97de835d-0f4e-474e-a583-987d6f6a4f2c/share/d8870ad0-1b10-454a-8c36-75350ea50cbb/call-atac/wf_atac/c53f4afa-8fd4-4995-93e1-7f4dee359afc/call-align/ENCSR023FME.atac.filter.fragments.hg38.tsv.gz', 'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/c467e890-b9f7-4458-a5b7-5581eb958cba/share/293acd9b-1e89-41d4-af76-247ca41af146/call-atac/wf_atac/2c759a0e-018b-4d6e-bbfc-257cbab95224/call-align/ENCSR868ZEI

In [15]:
len(gs_path_to_search_sample_id)

25

In [16]:
gs_atac_dataset = [x.split("/")[-1].split(".")[0] for x in gs_path_to_search_sample_id]

In [17]:
atac_dataset_list = df_encode_rows["atac_dataset"].tolist()
atac_dataset_list[0:3]

['ENCSR114OKU', 'ENCSR114OKU', 'ENCSR114OKU']

In [18]:
# atac dataset in both
atac_dataset_to_fragment = list(set(gs_atac_dataset) & set(atac_dataset_list))
atac_dataset_to_fragment[0:6]

['ENCSR868ZEI',
 'ENCSR848EID',
 'ENCSR367GKP',
 'ENCSR660NEE',
 'ENCSR261TQO',
 'ENCSR397LYX']

In [19]:
len(atac_dataset_to_fragment)


23

In [20]:
# this will check for which fragments do we have annotation data
df_relevant_rows = df_encode_rows[df_encode_rows['atac_dataset'].isin(atac_dataset_to_fragment)]
df_relevant_rows.head(2)

,cell_type_name,rna_dataset,rna_library,rna_barcode,atac_dataset,atac_library,atac_barcode,file
510368,macrophage,ENCSR231FNL,ENCLB398IAZ,AAACCGCGTGACATAT,ENCSR987PQH,ENCLB714RNW,GTCACCATGAGCTATT,EAID_000093
510369,cardiomyocyte,ENCSR231FNL,ENCLB398IAZ,AAACCGCGTTACATCC,ENCSR987PQH,ENCLB714RNW,ATGTTGTTGAGCTATT,EAID_000093


In [21]:
gs_path_to_fragment = list(filter(lambda x: any(substring in x for substring in atac_dataset_to_fragment), gs_path_to_search_sample_id))
print(gs_path_to_fragment[0:4])

['gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/0e8c4f76-5f9f-4dc8-8ef3-608afb9395ab/share/38144a43-b98a-4c90-93ff-a7fbb88efd2f/call-atac/wf_atac/e7b3e84a-fea9-4fe6-ad7a-40af393fdb36/call-align/ENCSR987PQH.atac.filter.fragments.hg38.tsv.gz', 'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/97de835d-0f4e-474e-a583-987d6f6a4f2c/share/d8870ad0-1b10-454a-8c36-75350ea50cbb/call-atac/wf_atac/c53f4afa-8fd4-4995-93e1-7f4dee359afc/call-align/ENCSR023FME.atac.filter.fragments.hg38.tsv.gz', 'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/c467e890-b9f7-4458-a5b7-5581eb958cba/share/293acd9b-1e89-41d4-af76-247ca41af146/call-atac/wf_atac/2c759a0e-018b-4d6e-bbfc-257cbab95224/call-align/ENCSR868ZEI.atac.filter.fragments.hg38.tsv.gz', 'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/ff3a5809-0cd2-464c-8a29-9bf6311111ae/share/02c2df11-8d6c-4786-868a-94ecf3858bc5/call-atac/wf_atac/26b48087-2f17-469d-b3ff-b64720f9ddb4/call-align/ENCSR367GKP

In [34]:
gs_path_to_fragment[0]

'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/0e8c4f76-5f9f-4dc8-8ef3-608afb9395ab/share/38144a43-b98a-4c90-93ff-a7fbb88efd2f/call-atac/wf_atac/e7b3e84a-fea9-4fe6-ad7a-40af393fdb36/call-align/ENCSR987PQH.atac.filter.fragments.hg38.tsv.gz'

In [35]:
gs_path_to_fragment[1]

'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/97de835d-0f4e-474e-a583-987d6f6a4f2c/share/d8870ad0-1b10-454a-8c36-75350ea50cbb/call-atac/wf_atac/c53f4afa-8fd4-4995-93e1-7f4dee359afc/call-align/ENCSR023FME.atac.filter.fragments.hg38.tsv.gz'

In [22]:
len(gs_path_to_fragment)

23

In [23]:
gs_path_to_fragment[0]

'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/0e8c4f76-5f9f-4dc8-8ef3-608afb9395ab/share/38144a43-b98a-4c90-93ff-a7fbb88efd2f/call-atac/wf_atac/e7b3e84a-fea9-4fe6-ad7a-40af393fdb36/call-align/ENCSR987PQH.atac.filter.fragments.hg38.tsv.gz'

In [24]:
gs_path_to_fragment[1]

'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/97de835d-0f4e-474e-a583-987d6f6a4f2c/share/d8870ad0-1b10-454a-8c36-75350ea50cbb/call-atac/wf_atac/c53f4afa-8fd4-4995-93e1-7f4dee359afc/call-align/ENCSR023FME.atac.filter.fragments.hg38.tsv.gz'

In [25]:
gs_path_to_fragment[2]

'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/c467e890-b9f7-4458-a5b7-5581eb958cba/share/293acd9b-1e89-41d4-af76-247ca41af146/call-atac/wf_atac/2c759a0e-018b-4d6e-bbfc-257cbab95224/call-align/ENCSR868ZEI.atac.filter.fragments.hg38.tsv.gz'

In [30]:
# os.path.exists(full_local_fragment_file_name)

False

In [33]:
# %run tsv_files_utils.ipynb

# List of column names
pipeline_column_names = ['dataset_id_cell_id', 'cell_type_id', 'gs_path','chromosome']
# Initialize dataframe with column names
df_pseudobulk_pipeline= pd.DataFrame(columns=pipeline_column_names)


# Tanjin, here you decide how many samples to run
# where you decide to run on samples [0:1] or more
for gs_fragment_file in gs_path_to_fragment[1:2]:
    print("!!!!!gs_fragment_file is {}".format(gs_fragment_file))
#     if ("ENCSR023FME" not in gs_fragment_file):
#         continue
    cur_atac_dataset = gs_fragment_file.split("/")[-1].split(".")[0]
    print("cur_atac_dataset is {}".format(cur_atac_dataset))
    
    # Step 1: input: labels file. output: df of the subset by cell_type_id
    # Step 1-1: get the labels file from synpase.
    dataset_id = 'syn34271785'
    folder_name = df_relevant_rows[df_relevant_rows['atac_dataset']==cur_atac_dataset]['file'].to_list()[0]
    print("folder_name in synpase is {}".format(folder_name))
    # (starting_folder_id, first_subfolder_name, second_subfolder_name):
    file_id_list = get_file_names_in_second_subfolder(dataset_id,folder_name,"labels")
    file_id = file_id_list[0] # the label's file synID
    print('file_id is {}'.format(file_id))
    
    # Download the synpase label file:
    # Obtain a pointer and download the data to full_labels_file_path
    labels_file_path = os.path.join(local_folder,"{}_labels".format(file_id))
    label_file_synHandle = syn.get(entity=file_id,downloadLocation=labels_file_path) 
    print('label_file_synHandle is {}'.format(label_file_synHandle))
    full_labels_file_path = label_file_synHandle.path
#     /home/jupyter/eila/syn46645403_labels/cell_types.tsv.gz
    print('full_labels_file_path is {}'.format(full_labels_file_path))
    
#     # Load the labels data to dataframe. has all the barcodes and the cell types relevant for them
    df_labels = read_tsv_gz_to_dataframe_skipping_comments_and_empty_lines(full_labels_file_path)
    cell_type_id_names = list(set(df_labels['cell_type_id'].to_list()))
    print("number of cell_type_id_names is {}".format(len(cell_type_id_names)))

    
    # Stream file fragment 
    print("!!!gs fragment_file is {}".format(gs_fragment_file))
    local_fragment_file_name = os.path.basename(gs_fragment_file)
    full_local_fragment_file_name = os.path.join(local_folder,local_fragment_file_name)
    print("full_local_fragment_file_name is {}".format(full_local_fragment_file_name))
    
    if (os.path.exists(full_local_fragment_file_name) == False):
      print("!!!!gsutil cp {} to {}".format(gs_fragment_file,full_local_fragment_file_name))
      !gsutil cp $gs_fragment_file $full_local_fragment_file_name
      print("!!!!finished cp {} to {}".format(gs_fragment_file,full_local_fragment_file_name))
    else:
      print("full_local_fragment_file_name: {} exists. nothign to copy".format(full_local_fragment_file_name))
    
    # logis for tagAlign
    for cell_type_id in cell_type_id_names: # all the cell types ids avaiable in synpase file
      # the output fragmenf file
      tagAlign_file_name = "tagAlign_{}_{}_fragment_file.tsv.gz".format(cur_atac_dataset,cell_type_id)
      print("tagAlign_file_name is {}".format(tagAlign_file_name))
      full_path_cell_type_fragment_tagAlign_file_name = os.path.join(os.path.dirname(full_local_fragment_file_name),tagAlign_file_name)
      print("full_path_cell_type_fragment_tagAlign_file_name is {}".format(full_path_cell_type_fragment_tagAlign_file_name))
        
      # get the cell barcode of the specific cell type: Basal_epithelial_cell
      cells_barcodes_for_cell_type = df_labels[df_labels['cell_type_id'] ==cell_type_id]
      print("cells_barcodes_for_cell_type[0:5] is {}".format(cells_barcodes_for_cell_type[0:5])) 
      print("len(cells_barcodes_for_cell_type) is {}".format(len(cells_barcodes_for_cell_type))) 
     
      gs_dest_tagAlign = os.path.join(os.path.dirname(gs_fragment_file),"tagAlign",tagAlign_file_name)
      print("gs_dest_tagAlign is  {} ".format(gs_dest_tagAlign))

      tagAlignFile_exists = !gsutil ls $gs_dest_tagAlign
      tagAlignFile_exists = tagAlignFile_exists[0]
      if (os.path.basename(gs_dest_tagAlign) in tagAlignFile_exists):
        print("gs_dest_tagAlign exists. {}. Nothing to do".format(gs_dest_tagAlign))
        # I can add here remove the local copy of the file
        continue #
      
      # in any case, remove the local file that was created from previous executions
      print("remove local full_path_cell_type_fragment_tagAlign_file_name {}".format(full_path_cell_type_fragment_tagAlign_file_name))
      remove_file(full_path_cell_type_fragment_tagAlign_file_name)

      print("!!!!!open full_local_fragment_file_name {}".format(full_local_fragment_file_name))
      with gzip.open(full_local_fragment_file_name, "rt") as i, gzip.open(full_path_cell_type_fragment_tagAlign_file_name, 'wt') as o:
        num_of_lines_written=0
        for line_number, line in enumerate(i, start=1):
#           print("line is {}".format(line))
          print("line_number is {}".format(line_number))
          out_list = split_fragment_line_string(line)
#         out_list is ['chr1', '10043', '10113', 'TCTATTCCTGTCGCGGAAAGTC_ENCSR023FME', 'TCTATTCCTGTCGCGGAAAGTC', 'ENCSR023FME', '1', 'ENCSR023FME_TCTATTCCTGTCGCGGAAAGTC']
#           print("out_list is {}".format(out_list))
          bc = out_list[-1]
          print("bc is {}".format(bc))
#           print("cell_type_id is {}".format(cell_type_id))
        
          
          cell_barcodes_for_current_cell_type = cells_barcodes_for_cell_type['cell_id'].to_list()
          if bc in cell_barcodes_for_current_cell_type:
            print("!!!bc is {}. cell_barcodes_for_current_cell_type[0:5] is {}".format(bc,cell_barcodes_for_current_cell_type[0:5]))
#             Austin output: chrom, start, end, bc, rem = line.rstrip('\n').split('\t', 5)
            out_line_to_print = f"{out_list[0]}\t{out_list[1]}\t{out_list[2]}\t{bc}\t{out_list[-2]}\n"
#             chr1	10007	10175	ENCSR023FME#ENCSR023FME_GAAGGTTCAAAGTGTCAGTCAA	1
            print("out_line_to_print is {}".format(out_line_to_print))
            num_of_lines_written +=1
            returnTagAlign = convert_fragment_line_to_tagAlign(out_line_to_print)
            print("!!!!out_list \n {} was converted to \n {}".format(out_list,returnTagAlign))
            o.write(returnTagAlign)
            print("returnTagAlign is : {} to file {}".format(returnTagAlign,full_path_cell_type_fragment_tagAlign_file_name))
        
            # for debug
            # Break the loop after 10 lines
#           if line_number == 100:
#             break
        print("finished writing to full_path_cell_type_fragment_tagAlign_file_name {}. num_of_lines_written is {}".format(full_path_cell_type_fragment_tagAlign_file_name,num_of_lines_written))
 
        print("num_of_lines_written is {}".format(num_of_lines_written))
        if (num_of_lines_written ==0):
          continue

#       !gsutil -m cp $full_path_cell_type_fragment_tagAlign_file_name $gs_dest_tagAlign
      print("finish copy tagAlign file {} to gcp  {}".format(full_path_cell_type_fragment_tagAlign_file_name,gs_dest_tagAlign))
        
#       # chromosome_size_human - taken from the worskapce dashboard
      df_pseudobulk_pipeline = pd.concat([df_pseudobulk_pipeline, \
                                          pd.DataFrame({'dataset_id_cell_id': [tagAlign_file_name.strip(".tsv")], \
                                                        'cell_type_id': [cell_type_id], \
                                                        'gs_path':[gs_dest_tagAlign], \
                                                        'chromosome': [gs_chrom_size_human]})], \
                                                        ignore_index=True)
        
      print("!!!!df_pseudobulk_pipeline is {} ".format(df_pseudobulk_pipeline))

# # save the pipeline input table
local_pseudobulk_pipeline_table = os.path.join(local_folder,"{}_pipeline_table.tsv".format(cur_atac_dataset))
print("local_pseudobulk_pipeline_table is {}".format(local_pseudobulk_pipeline_table))
df_pseudobulk_pipeline.to_csv(local_pseudobulk_pipeline_table, sep='\t', index=False)
print("finsihed copy {}".format(local_pseudobulk_pipeline_table))
gs_pseudobulk_pipeline_table = os.path.join("gs://",bucket_id,gs_path_for_the_encode_pseudobulk_pipeline_table)
print("gs_pseudobulk_pipeline_table is  {}".format(gs_pseudobulk_pipeline_table))
!gsutil cp $local_pseudobulk_pipeline_table $gs_pseudobulk_pipeline_table
print("finished copying to {}".format(gs_pseudobulk_pipeline_table))


!!!!!gs_fragment_file is gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/97de835d-0f4e-474e-a583-987d6f6a4f2c/share/d8870ad0-1b10-454a-8c36-75350ea50cbb/call-atac/wf_atac/c53f4afa-8fd4-4995-93e1-7f4dee359afc/call-align/ENCSR023FME.atac.filter.fragments.hg38.tsv.gz
cur_atac_dataset is ENCSR023FME
folder_name in synpase is EAID_000011
get_file_names parameters are: folder_id syn34271785 and current_depth 0
current_depth is 0 
folder_entities are <generator object Synapse.getChildren at 0x29fa7f640> 
get_file_names parameters are: folder_id syn46645395 and current_depth 1
current_depth is 1 
folder_entities are <generator object Synapse.getChildren at 0x2acd64040> 
get_file_names parameters are: folder_id syn46645400 and current_depth 2
folder_entities are <generator object Synapse.getChildren at 0x2acd64160>
file_id is syn46645403
label_file_synHandle is File: cell_types.tsv.gz (syn46645403)
  md5=c4e639ecf918b07c4a1c93e17ac34f23
  fileSize=59179
  contentType=text/tab-se

FileNotFoundError: [Errno 2] No such file or directory: '/Users/eilaarich-landkof-stanford/Documents/Code/git_kundaje_genomics/DNA/ATAC/single-cell/fragments-file-manipulation/ENCSR023FME.atac.filter.fragments.hg38.tsv.gz'